# Exploratory Data Analysis for Dengue Competition
competition link: https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/data/
completed on 4 Feb 2020

trying out splitting by cities, as well as using select features
insights from benchmark: http://drivendata.co/blog/dengue-benchmark/


In [1]:
import numpy as np
import pandas as pd

In [2]:
test_features = pd.read_csv("dengue_features_test.csv", index_col=[0,1,2]) 
train_features = pd.read_csv("dengue_features_train.csv", index_col=[0,1,2]) 
train_labels = pd.read_csv("dengue_labels_train.csv", index_col=[0,1,2]) 
submission_format = pd.read_csv("submission_format.csv", index_col=[0,1,2]) 

In [3]:
train_features

week_start_date   ndvi_ne   ndvi_nw   ndvi_se   ndvi_sw  \
city year weekofyear                                                           
sj   1990 18              1990-04-30  0.122600  0.103725  0.198483  0.177617   
          19              1990-05-07  0.169900  0.142175  0.162357  0.155486   
          20              1990-05-14  0.032250  0.172967  0.157200  0.170843   
          21              1990-05-21  0.128633  0.245067  0.227557  0.235886   
          22              1990-05-28  0.196200  0.262200  0.251200  0.247340   
...                              ...       ...       ...       ...       ...   
iq   2010 21              2010-05-28  0.342750  0.318900  0.256343  0.292514   
          22              2010-06-04  0.160157  0.160371  0.136043  0.225657   
          23              2010-06-11  0.247057  0.146057  0.250357  0.233714   
          24              2010-06-18  0.333914  0.245771  0.278886  0.325486   
          25              2010-06-25  0.298186  0.232971  0.274214  0.315757   

                      precipitation_amt_mm  reanalysis_air_temp_k  \
city year weekofyear                                                
sj   1990 18                         12.42             297.572857   
          19                         22.82             298.211429   
          20                         34.54             298.781429   
          21                         15.36             298.987143   
          22                          7.52             299.518571   
...                                    ...                    ...   
iq   2010 21                         55.30             299.334286   
          22                         86.47             298.330000   
          23                         58.94             296.598571   
          24                         59.67             296.345714   
          25                         63.22             298.097143   

                      reanalysis_avg_temp_k  reanalysis_dew_point_temp_k  \
city year weekofyear                                                       
sj   1990 18                     297.742857                   292.414286   
          19                     298.442857                   293.951429   
          20                     298.878571                   295.434286   
          21                     299.228571                   295.310000   
          22                     299.664286                   295.821429   
...                                     ...                          ...   
iq   2010 21                     300.771429                   296.825714   
          22                     299.392857                   296.452857   
          23                     297.592857                   295.501429   
          24                     297.521429                   295.324286   
          25                     299.835714                   295.807143   

                      reanalysis_max_air_temp_k  ...  \
city year weekofyear                             ...   
sj   1990 18                              299.8  ...   
          19                              300.9  ...   
          20                              300.5  ...   
          21                              301.4  ...   
          22                              301.9  ...   
...                                         ...  ...   
iq   2010 21                              309.7  ...   
          22                              308.5  ...   
          23                              305.5  ...   
          24                              306.1  ...   
          25                              307.8  ...   

                      reanalysis_precip_amt_kg_per_m2  \
city year weekofyear                                    
sj   1990 18                                    32.00   
          19                                    17.94   
          20                                    26.10   
          21                                    13.90   
          22          

In [4]:

# Separate data for Iquitos
iq_train_features = train_features.loc["iq"]
iq_train_labels = train_labels.loc["iq"]

# Seperate data for San Juan
sj_train_features = train_features.loc['sj']
sj_train_labels = train_labels.loc["sj"]


In [17]:
# Separate data for Iquitos
iq_test_features = test_features.loc["iq"]

# Seperate data for San Juan
sj_test_features = test_features.loc['sj']

# Gradient Boosting Model, SJ (XGBoost)

model, without using city and week date

In [5]:
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [18]:
X = sj_train_features.drop(columns = [ "week_start_date"])
Y = sj_train_labels["total_cases"]

In [19]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [20]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [21]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [22]:
# evaluate predictions
accuracy = mean_absolute_error(y_test, predictions)
print("MAE: %.2f%%" % (accuracy))

MAE: 25.16%


MAE: 25.16%

so high

## submission 

In [41]:
sj_test = sj_test_features.drop(columns = ["week_start_date"])
sj_test_pred = model.predict(sj_test)
sj_test_predictions = [round(value) for value in sj_test_pred]

sj_submission = submission_format.loc["sj"]
sj_submission["total_cases"] = sj_test_predictions
sj_submission = sj_submission.reset_index()
sj_submission.insert(0, 'city', 'sj')
sj_submission

,city,year,weekofyear,total_cases
0,sj,2008,18,30
1,sj,2008,19,3
2,sj,2008,20,38
3,sj,2008,21,5
4,sj,2008,22,3
...,...,...,...,...
255,sj,2013,13,5
256,sj,2013,14,5
257,sj,2013,15,8
258,sj,2013,16,8


# Gradient Boosting Model, IQ (XGBoost)

model, without using city and week date

In [27]:
X = iq_train_features.drop(columns = ["week_start_date"])
Y = iq_train_labels["total_cases"]

In [28]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [29]:
# fit model no training data
iq_model = XGBClassifier()
iq_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [30]:
# make predictions for test data
y_pred = iq_model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [31]:
# evaluate predictions
accuracy = mean_absolute_error(y_test, predictions)
print("MAE: %.2f%%" % (accuracy))

MAE: 6.28%


MAE 6.28% pretty low

## submission

In [42]:
iq_test = iq_test_features.drop(columns = ["week_start_date"])
iq_test_pred = model.predict(iq_test)
iq_test_predictions = [round(value) for value in iq_test_pred]

iq_submission = submission_format.loc["iq"]
iq_submission["total_cases"] = iq_test_predictions
iq_submission = iq_submission.reset_index()
iq_submission.insert(0, 'city', 'iq')

iq_submission

,city,year,weekofyear,total_cases
0,iq,2010,26,24
1,iq,2010,27,24
2,iq,2010,28,71
3,iq,2010,29,24
4,iq,2010,30,71
...,...,...,...,...
151,iq,2013,22,24
152,iq,2013,23,24
153,iq,2013,24,24
154,iq,2013,25,24


## Submission on 7th Feb 2020

In [43]:
submission = sj_submission.append(iq_submission)

In [44]:
submission

,city,year,weekofyear,total_cases
0,sj,2008,18,30
1,sj,2008,19,3
2,sj,2008,20,38
3,sj,2008,21,5
4,sj,2008,22,3
...,...,...,...,...
151,iq,2013,22,24
152,iq,2013,23,24
153,iq,2013,24,24
154,iq,2013,25,24


In [45]:
submission.to_csv("submission_citysplit.csv", index = False)

In [ ]:
only_cases = submission["total_cases"]
only_cases.to_csv("submission_xgboost_4.csv", header = True)

In [ ]:
only_cases

In [ ]:
submission